<a href="https://colab.research.google.com/github/W2NJL/CSC8490/blob/main/CSC8490project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Set up the Oracle client - as neatly prepared by Prof. Frein Esq.

! pip install cx_Oracle

! ls -l oracle-instantclient*-basiclite-*.rpm || wget https://yum.oracle.com/repo/OracleLinux/OL7/oracle/instantclient/x86_64/getPackage/oracle-instantclient19.3-basiclite-19.3.0.0.0-1.x86_64.rpm

! ls -l oracle-instantclient*-sqlplus-*.rpm   || wget https://yum.oracle.com/repo/OracleLinux/OL7/oracle/instantclient/x86_64/getPackage/oracle-instantclient19.3-sqlplus-19.3.0.0.0-1.x86_64.rpm

! sudo apt-get install alien libaio1

! sudo alien -i oracle-instantclient19.3-basiclite-19.3.0.0.0-1.x86_64.rpm

! sudo alien -i oracle-instantclient19.3-sqlplus-19.3.0.0.0-1.x86_64.rpm

%env ORACLE_HOME=/usr/lib/oracle/19.3/client64

%env LD_LIBRARY_PATH=/usr/lib/oracle/19.3/client64/lib

print('Setup finished!')

In [ ]:
# connect into Nick's Oracle DB instance on Prof. Frein's cloud server
import numpy as np
import pandas as pd
import cx_Oracle

CONN_INFO = {
 'host': '45.79.135.253',
 'port': '1521',
 'user': 'nlanga01',
 'psw': '00700368',
 'service': 'xe',
}
CONN_STR = '{user}/{psw}@{host}:{port}/{service}'.format(**CONN_INFO)
connection = cx_Oracle.connect(CONN_STR)

cursor = connection.cursor()

print(connection.version)

In [ ]:
# SELECT QUERY:  Which stations, in the LOGPROPMETHOD table, have been logged by multiple propagation methods, how many times total, and how many times have they been logged in each method?
# I also would like to see what frequency those stations are on, and their city and state of license, from the NORTHAMERICANFM table.
# The propagation methods are tropospheric ducting, E-Skip, and meteor scatter.

results = []

for result in cursor.execute("SELECT n.CallLetters, n.Frequency, n.City, n.State,  COUNT(*) as total, \
    SUM(CASE WHEN l.PropMethod = 'Tr' THEN 1 ELSE 0 END) as Tropo, \
    SUM(CASE WHEN l.PropMethod = 'Es' THEN 1 ELSE 0 END) as E_Skip, \
    SUM(CASE WHEN l.PropMethod = 'Ms' THEN 1 ELSE 0 END) as Meteor_Scatter \
FROM logpropmethod l, NORTHAMERICANFM n \
WHERE n.CALLLETTERS = l.CALLLETTERS \
GROUP BY n.CallLetters, n.Frequency, n.City, n.State \
HAVING COUNT(*) > 1"):
  results.append(result)

# Because I could not resist, I had to figure out the way for the column names to print for this query to somewhat make sense!     
column_names = []
data_rows = []
column_names = [desc[0] for desc in cursor.description]
for row in cursor:
  data_rows.append(row)
    
# Print the results of the query
print("{}".format(column_names))
for i in results:
  print (i)



In [ ]:
# STORED PROCEDURE:
# This procedure prompts you, our dear user, for the call letters of a licensed North American FM radio station
# After entering the call letters, if the call letters are in the NORTHAMERICANFM table (the table is not 100% comprehensive), it will calculate the distance via its coordinates
# between the station's transmitter and the coordinates for Mendel Hall at Villanova University.  You can use call letters such as WIP (for 94.1 WIP in Philadelphia) as an example, 
# or Villanova's own WXVU (shameless plug - for which I am the music director!).
# The distance is returned in miles in decimal format (i.e 5.6)
# The location portion of the procedure input I have as a variable so it can be customized to refer to coordinates for a given location (you can substitute 'Villanova' for 'Nick' for my home coordinates).
result = cursor.var(cx_Oracle.NUMBER)

location = "Villanova"
val = input("Enter the callletters of the station you wish to find the distance in miles from Villanova University: ") 
# calling the procedure
cursor.callproc("FINDDISTANCE", [location, val, result])
res = result.getvalue()
print("Distance from " + location + " to " + val + " is: {}".format(res) + " miles")



In [ ]:
# CHANGE SOME DATA:
# Update statement:  In my DXPEDITIONS table, I set aside a derived attribute 'AMOUNTLOGGED' to show the amount of stations logged at each DXpedition event.  
# The stations logged exist in the 'DXPEDITIONLOGS' table.  I used an Update statement to calculate the count of logs from each DXpedition, then place it in its respective
# row in in the DXPEDITIONS table.  Here, I will print the DXPEDITIONS table as is (with the 'AMOUNTLOGGED' attribute null), then run the update statement, then print the subsequent select statement
# in the next block.

#FIRST, let's set the 'AMOUNTLOGGED' column to null (in case I ran this Update statement recently!).
reset_query = "UPDATE DXPEDITION SET AMOUNTLOGGED = null"
cursor.execute(reset_query)

#BEFORE (you will see a 'None' result at the end of the query, which represents the 'AMOUNTLOGGED' attribute):
for result in cursor.execute("SELECT * from DXPEDITION"):
    print(result)
#UPDATE STATEMENT:
sql_update_query = "update dxpedition set amountlogged = (select count (*) from dxpeditionlogs where dxpeditionlogs.dxpedition = dxpedition.name)"
cursor.execute(sql_update_query)

In [ ]:
#AFTER (there is only one DXPEDITION loaded in the DXPEDITION table at this time)
for result in cursor.execute("SELECT * from DXPEDITION"):
    print(result)

In [ ]:
 #make changes public and permanent
 connection.commit()

In [ ]:
#Be kind, close the connection
connection.close()